In [1]:
import pandas as pd
import yfinance as yf


In [2]:
# Brazilian soybean monthly production dataset

df = pd.read_csv('../raw_data/IBGE_Soja_mensal_completo.csv', index_col = 0, parse_dates=['Mes-Ano'])
df

,Mes-Ano,Área plantada (Hectares),Área colhida (Hectares),Produção (Toneladas),Rendimento médio (Quilogramas por Hectare)
0,2000-09-01,13655653,13619991,32688027,2400
1,2000-10-01,13655653,13619991,32687427,2400
2,2000-11-01,13655653,13619991,32693350,2400
3,2000-12-01,13653793,13618131,32679270,2400
4,2001-01-01,13618529,13610326,35103551,2579
...,...,...,...,...,...
253,2021-10-01,38866350,38819606,134086406,3454
254,2021-11-01,38898843,38852091,134254267,3456
255,2021-12-01,38988530,38974763,134933704,3462
256,2022-01-01,40391103,40391103,131796763,3263


In [3]:
# Creating a daily datetime column

lista_dias = pd.date_range(df['Mes-Ano'].values[0],df['Mes-Ano'].values[-1])
lista_dias

DatetimeIndex(['2000-09-01', '2000-09-02', '2000-09-03', '2000-09-04',
               '2000-09-05', '2000-09-06', '2000-09-07', '2000-09-08',
               '2000-09-09', '2000-09-10',
               ...
               '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26',
               '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30',
               '2022-01-31', '2022-02-01'],
              dtype='datetime64[ns]', length=7824, freq='D')

In [4]:
#Creating a daily datetime dataframe

lista_dias_df = pd.DataFrame(lista_dias, columns=['Dia'])
lista_dias_df

,Dia
0,2000-09-01
1,2000-09-02
2,2000-09-03
3,2000-09-04
4,2000-09-05
...,...
7819,2022-01-28
7820,2022-01-29
7821,2022-01-30
7822,2022-01-31


In [5]:
#Creating a month column according to the related day

lista_dias_df['Mes-Ano'] = lista_dias_df['Dia'].apply(lambda data: pd.to_datetime(f'{data.month}-{data.year}'))

In [6]:
lista_dias_df

,Dia,Mes-Ano
0,2000-09-01,2000-09-01
1,2000-09-02,2000-09-01
2,2000-09-03,2000-09-01
3,2000-09-04,2000-09-01
4,2000-09-05,2000-09-01
...,...,...
7819,2022-01-28,2022-01-01
7820,2022-01-29,2022-01-01
7821,2022-01-30,2022-01-01
7822,2022-01-31,2022-01-01


In [7]:
# Merging brazilian soybean production dataset with daily dataset
# Rename 'Dia' column by "Date"
# Drop columns 'Área plantada (Hectares)', 'Área colhida (Hectares)' and 'Rendimento médio (Quilogramas por Hectare)'

novo_df = df.merge(lista_dias_df,on='Mes-Ano')
novo_df.rename(columns = {'Dia':'Date','Produção (Toneladas)':'Expectativa Produção Safra Anual (Ton)'}, inplace = True)
novo_df.drop(columns=['Área plantada (Hectares)','Área colhida (Hectares)','Rendimento médio (Quilogramas por Hectare)'], inplace=True)
novo_df

,Mes-Ano,Expectativa Produção Safra Anual (Ton),Date
0,2000-09-01,32688027,2000-09-01
1,2000-09-01,32688027,2000-09-02
2,2000-09-01,32688027,2000-09-03
3,2000-09-01,32688027,2000-09-04
4,2000-09-01,32688027,2000-09-05
...,...,...,...
7819,2022-01-01,131796763,2022-01-28
7820,2022-01-01,131796763,2022-01-29
7821,2022-01-01,131796763,2022-01-30
7822,2022-01-01,131796763,2022-01-31


In [8]:
# Using yfinance to retrieve data
# Soybean_ticker = 'ZS=F'
# Soybean_futures_prices = 'zsf'

zsf = yf.Ticker("ZS=F")

# Get stock info
zsf.info

# Get historical market data
hist_soybean_df = zsf.history(start = '2000-09-15', end='2022-03-22')
hist_soybean_df = hist_soybean_df.drop(columns=['Dividends','Stock Splits'])

In [9]:
hist_F = hist_soybean_df.reset_index()
hist_F

,Date,Open,High,Low,Close,Volume
0,2000-09-15,491.00,493.75,490.00,493.75,445
1,2000-09-18,483.25,486.00,482.50,484.00,382
2,2000-09-19,485.00,485.50,483.50,484.25,325
3,2000-09-20,483.50,484.25,483.25,483.25,321
4,2000-09-21,486.75,488.00,486.00,486.50,330
...,...,...,...,...,...,...
5407,2022-03-15,1674.00,1674.00,1638.00,1658.75,70449
5408,2022-03-16,1661.00,1681.25,1645.25,1649.25,65232
5409,2022-03-17,1649.25,1677.50,1647.25,1668.50,74317
5410,2022-03-18,1669.75,1689.00,1651.50,1668.00,67110


In [10]:
# Merging brazilian soybean production dataset with soybean daily price dataset

df_preco_producao_BR = pd.merge(novo_df, hist_F,how = 'left', on = 'Date')
df_preco_producao_BR.tail(100)

,Mes-Ano,Expectativa Produção Safra Anual (Ton),Date,Open,High,Low,Close,Volume
7724,2021-10-01,134086406,2021-10-25,1218.75,1238.25,1216.00,1237.25,137468.0
7725,2021-10-01,134086406,2021-10-26,1236.25,1246.25,1230.00,1238.00,94349.0
7726,2021-10-01,134086406,2021-10-27,1237.25,1257.50,1227.00,1239.25,122469.0
7727,2021-10-01,134086406,2021-10-28,1238.50,1249.50,1229.50,1233.75,73201.0
7728,2021-10-01,134086406,2021-10-29,1230.25,1242.00,1227.75,1235.75,10990.0
...,...,...,...,...,...,...,...,...
7819,2022-01-01,131796763,2022-01-28,1446.50,1479.00,1441.50,1470.00,143879.0
7820,2022-01-01,131796763,2022-01-29,NaN,NaN,NaN,NaN,NaN
7821,2022-01-01,131796763,2022-01-30,NaN,NaN,NaN,NaN,NaN
7822,2022-01-01,131796763,2022-01-31,1474.00,1496.75,1473.50,1490.50,148673.0
